In [3]:
import pandas as pd
import re
from joblib import Parallel, delayed

In [4]:
#DATOS
train_reviews=pd.read_csv('train_reviews.csv', sep=',')
test_reviews = pd.read_csv('test_reviews.csv')
negocios_df=pd.read_csv('negocios.csv')

df = train_reviews.merge(negocios_df, on='business_id',how='inner')
df

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,ZZO43qKB-s65zplC8RfJqw,-1BSu2dt_rOAqllw9ZDXtA,smkZq4G1AOm4V6p3id5sww,5.0,0,0,0,Fantastic fresh food. The greek salad is amazi...,2016-09-30 15:49:32,The Hungry Greek,...,FL,33626,28.069486,-82.633828,4.5,142,1,"{'Caters': 'True', 'NoiseLevel': ""u'average'"",...","Restaurants, Greek","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
1,vojXOF_VOgvuKD95gCO8_Q,xpe178ng_gj5X6HgqtOing,96_c_7twb7hYRZ9HHrq01g,1.0,2,0,1,Been a patient at Largo Med/Diagnostic Clinic ...,2020-12-09 14:39:51,Largo Medical Center,...,FL,33770,27.915015,-82.803637,2.5,22,1,"{'ByAppointmentOnly': 'True', 'AcceptsInsuranc...","Doctors, Medical Centers, Health & Medical, We...",NaN
2,KwxdbiseRlIRNzpgvyjY0Q,axbaerf2Fk92OB4b9_peVA,e0AYjKfSF0DL-5C1CpOq6Q,4.0,0,0,0,The location is convenient to my campus so I d...,2013-09-04 16:19:51,Crown Vision Center,...,MO,63108,38.641461,-90.246127,2.5,7,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Eyewear & Opticians, Laser Eye Surgery/Lasik, ...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-17:0', '..."
3,3mwoBcTy-2gMh0L91uaIeA,_GOiybb0rImYKJfwyxEaGg,vF-uptiQ34pVLHJKzPHUlA,5.0,0,0,0,I agree with all the other compliments posted ...,2019-03-02 12:24:14,My Greek Corner,...,FL,34677,28.049869,-82.698361,4.5,73,0,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Restaurants, Mediterranean, Greek","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
4,XfWf7XsBWs3kYyYq7Ns1ZQ,ojWKg3B5pH3ncAsxun3kUw,X28XK71RuEXPapeyUOwNzg,5.0,10,4,7,"Wanting to help out the local economy, I thoug...",2020-04-23 18:26:29,Pinellas Grill,...,FL,33782,27.859863,-82.728274,4.0,31,1,"{'Music': ""{'dj': False, 'background_music': F...","Bars, Restaurants, Nightlife, American (Tradit...","{'Monday': '8:0-22:30', 'Tuesday': '8:0-22:30'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967779,yhyJUBAJUG-9gEIQ7gXS_g,iZAAjkPZ0sopzfajcfdOUg,2PvPsZ3KRFCtHbQkNHvGpg,5.0,1,0,0,Ordered takeout and this place didn't disappoi...,2020-04-19 15:42:12,Asian Sofrito,...,AZ,85719,32.252991,-110.960560,4.5,147,0,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Restaurants, Caribbean, Asian Fusion, Fast Foo...","{'Monday': '11:0-21:0', 'Wednesday': '11:0-21:..."
967780,5NxUmqweVTAZ_-FGJywTYw,AN8XscFSH1jctLSqAQ9-bA,-K0zTgGyxo-AeSkcV0IVaA,4.0,0,0,0,I took our annual managers meeting there for d...,2014-02-10 18:27:03,Spiro's Restaurant,...,MO,63017,38.678176,-90.499149,4.5,158,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Mediterranean, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
967781,-0rEInvO7q5cSI81NlH3-g,qtgODIPIsKsH2j3rItF9Tw,eZCt3doDaA-l4sg_OM67YQ,5.0,0,0,1,My favorite local coffee shop! Great drinks (i...,2019-07-24 14:19:52,PROVIDER,...,IN,46202,39.788376,-86.139458,4.5,292,1,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...","Bakeries, Bars, Nightlife, Coffee & Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '8:0-16:0', '..."
967782,8icm1hiV87OZbDsyjQDLAQ,gu7nU1IM7U3lLwUbqLeiDQ,ww3YJXu5c18aGZXWmm00qg,5.0,0,0,0,We hit a quiet time here on a very busy weeken...,2011-11-08 06:38:05,Café Amelie,...,LA,70116,29.959867,-90.062903,4.5,2756,1,"{'RestaurantsReservations': 'True', 'NoiseLeve...","Southern, American (New), Breakfast & Brunch, ...","{'Thursday': '17:0-21:0', 'Friday': '17:0-21:0..."


In [5]:
#PREPROCESS DATA 

df['text'] = df['text'].astype(str) + " " + df['categories'].astype(str)

def preprocess_text_parallel(text):
    return re.findall(r'\b[a-zA-Z]+\b', text.lower())

train_reviews['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in df['text'])
test_reviews['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in test_reviews['text'])


In [ ]:
#TEXT MODEL WORD2VEC
''''''
import gensim
from gensim.models import Word2Vec
model = Word2Vec(
    sentences=train_reviews['tokens'],
    vector_size=50,  
    window=5,
    min_count=5,  
    workers=4,  
    epochs=7 
)
model''''''

In [1]:
#model.save("word2vec_model.model")
import gensim
from gensim.models import Word2Vec
model = Word2Vec.load("word2vec_model.model")


In [6]:
#APPLY TEXT MODEL
def get_review_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return [0] * model.vector_size  # Vector de ceros si no hay palabras en el modelo
    return sum(vectors) / len(vectors)

train_reviews['vector'] = train_reviews['tokens'].apply(lambda x: get_review_vector(x, model))

X_train = list(train_reviews['vector'])
y_train = train_reviews['stars']

In [7]:
#OPTUNA WITH LOGISTIC REGRESSION
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Objective function for Optuna optimization
def objective(trial):
    # Hyperparameter search space
    C = trial.suggest_loguniform('C', 1e-5, 100)  # Regularization strength (inverse)
    solver = trial.suggest_categorical('solver', ['liblinear', 'lbfgs', 'newton-cg', 'saga'])
    max_iter = trial.suggest_int('max_iter', 1000, 5000, step=500)
    penalty = trial.suggest_categorical('penalty', ['l2'])

    # Define the Logistic Regression model with the hyperparameters
    classifier = LogisticRegression(C=C, solver=solver, max_iter=max_iter, penalty=penalty)

    # Fit the model
    classifier.fit(X_train, y_train)

    # Make predictions and evaluate the model
    predicted_stars = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predicted_stars)
    
    return accuracy

# If data is not split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and run the Optuna study
study = optuna.create_study(direction='maximize')  # Maximize accuracy
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Train with the best parameters
best_classifier = LogisticRegression(**best_params)
best_classifier.fit(X_train, y_train)

# Final predictions
final_predictions = best_classifier.predict(X_test)

# Evaluate the final model
final_accuracy = accuracy_score(y_test, final_predictions)
print(f"Final Accuracy: {final_accuracy}")


c:\Users\anny0\anaconda3\envs\rl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-17 21:58:55,672] A new study created in memory with name: no-name-10b64b26-e3f8-42b8-addb-85682935793e
C:\Users\anny0\AppData\Local\Temp\ipykernel_41932\96075322.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 100)  # Regularization strength (inverse)
[I 2025-04-17 22:00:35,639] Trial 0 finished with value: 0.6305119422185713 and parameters: {'C': 0.023473292139638564, 'solver': 'liblinear', 'max_iter': 4500, 'penalty': 'l2'}. Best is trial 0 with value: 0.6305119422185713.
C:\Users\anny0\AppData\Local\Temp\ipykernel_41932

Best hyperparameters: {'C': 14.867708330182724, 'solver': 'newton-cg', 'max_iter': 2500, 'penalty': 'l2'}
Final Accuracy: 0.6377191214990933


In [8]:
#CLASSIFIER LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=2500,C= 14.867708330182724,solver='newton-cg',penalty='l2')
classifier.fit(X_train, y_train)

test_reviews['vector'] = test_reviews['tokens'].apply(lambda x: get_review_vector(x, model))

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test)

submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})

submission_df

,review_id,stars
0,ieYPmCImINjPzTDFmEKBKA,5.0
1,QIkJ8fZ4yx_QaHahWWszAA,5.0
2,seR2KhblYMWg-k9zzN6aYA,5.0
3,BToo00Fi5pfJFA5MI2HM5g,5.0
4,FHJAzi1imodBit3RWK7zQA,1.0
...,...,...
414760,5VF1OSReEKAJS3z7lc0VRg,5.0
414761,N_yTSNrcGzGUSFPyIhAZ-g,5.0
414762,zQLW0mkqDtmseg8pl-zbNA,4.0
414763,DspB5yysBWRlFe01j_LzeA,5.0


In [9]:
submission_df.to_csv('prediction2.csv', index=False)

In [15]:
#CLASSIFIER RANDOMFOREST

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

test_reviews['vector'] = test_reviews['tokens'].apply(lambda x: get_review_vector(x, model))

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test)

submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})


submission_df

,review_id,stars
0,ieYPmCImINjPzTDFmEKBKA,5.0
1,QIkJ8fZ4yx_QaHahWWszAA,4.0
2,seR2KhblYMWg-k9zzN6aYA,5.0
3,BToo00Fi5pfJFA5MI2HM5g,5.0
4,FHJAzi1imodBit3RWK7zQA,1.0
...,...,...
414760,5VF1OSReEKAJS3z7lc0VRg,4.0
414761,N_yTSNrcGzGUSFPyIhAZ-g,5.0
414762,zQLW0mkqDtmseg8pl-zbNA,4.0
414763,DspB5yysBWRlFe01j_LzeA,5.0


In [ ]:
#XGBOOST CLASSIFIER

from xgboost import XGBClassifier
y_train_adj = y_train - 1

classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
classifier.fit(X_train, y_train_adj)

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test) + 1  


submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})


## ----------------------------------------------------------------------

In [28]:
#TEXT MODEL FAST TEXT
from gensim.models import FastText

# Entrenando un modelo de FastText
fasttext_model = FastText(sentences=train_reviews['tokens'], vector_size=50, window=5, min_count=5, epochs=10)

# Guardar el modelo
fasttext_model.save("fasttext_model.model")


[  2.435158     1.7426578   -2.2816732    1.4190265   -3.9866297
  -2.1381655    7.7173657    9.381285     2.412754    -2.319085
   4.2360425    0.4872235   -2.8740783    9.6010275    1.4816085
   3.8669336    1.1823233   -3.0029852    4.431468    -0.6778918
   0.37830508  -0.43311736   3.9101524   -4.5283966    4.254603
  -1.6915829   -2.277165     3.4268434   -2.8793123   -4.5364056
  -1.8741431   -1.1017144    1.8511803    6.0124116   -1.3969359
   3.8262832    5.128269     3.7838519   -5.7721014    2.4978418
  -1.3022603    3.4016862    2.2423437    5.395104     5.346601
   6.325267    -2.0887353   -0.16304673   1.5064793  -10.476946  ]


In [ ]:
def get_review_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return [0] * model.vector_size
    
    return sum(vectors) / len(vectors)

train_reviews['vector'] = train_reviews['tokens'].apply(lambda x: get_review_vector(x, fasttext_model))

X_train = list(train_reviews['vector'])
y_train = train_reviews['stars']

In [22]:
from xgboost import XGBClassifier
y_train_adj = y_train - 1

classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
classifier.fit(X_train, y_train_adj)

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test) + 1  


submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})


/opt/conda/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:36:18] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
submission_df.to_csv('prediction.csv', index=False)

## -------------------------------------------------

In [ ]:
usuarios=pd.read_csv('usuarios.csv', sep=',')

df = df.merge(usuarios, on='user_id',how='inner')
df

In [75]:
df2=df.drop(['review_id',	'user_id'	,'business_id','text', 'date', 'name_x','address','city','state','attributes','categories','name_y','elite','friends','hours','yelping_since','postal_code'],axis=1)
df2.rename(columns={'stars_x':'stars','name_y':'user_name','useful_x':'useful','funny_x':'funny','cool_x':'cool'}, inplace=True)
df2

,stars,useful,funny,cool,latitude,longitude,stars_y,review_count_x,is_open,review_count_y,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,5.0,0,0,0,28.069486,-82.633828,4.5,142,1,9,...,0,0,0,0,0,1,0,0,0,0
1,1.0,2,0,1,27.915015,-82.803637,2.5,22,1,31,...,0,0,0,0,0,0,0,0,0,0
2,4.0,0,0,0,38.641461,-90.246127,2.5,7,1,4,...,0,0,0,0,1,0,0,0,0,0
3,5.0,0,0,0,28.049869,-82.698361,4.5,73,0,55,...,0,0,0,0,0,0,1,1,0,0
4,5.0,10,4,7,27.859863,-82.728274,4.0,31,1,176,...,1,1,0,0,5,5,6,6,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967778,5.0,1,0,0,32.252991,-110.960560,4.5,147,0,50,...,0,0,0,0,0,3,0,0,0,0
967779,4.0,0,0,0,38.678176,-90.499149,4.5,158,1,210,...,0,0,0,0,0,2,0,0,0,1
967780,5.0,0,0,1,39.788376,-86.139458,4.5,292,1,5,...,0,0,0,0,1,0,0,0,0,0
967781,5.0,0,0,0,29.959867,-90.062903,4.5,2756,1,23,...,0,0,0,0,0,0,0,0,0,0


In [84]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
df2=df2[['funny','stars','cool','useful']]
# Crear un codificador para user_id y business_id
label_encoder = LabelEncoder()

# Ahora puedes usar estas nuevas columnas codificadas para el entrenamiento

# --------- 1. Prepara X e y ---------
y_train = df2['stars'].round().astype(int)  # Convertimos a clasificación (1–5 estrellas, por ejemplo)
X_train = df2.drop(columns=['stars'])  # postal_code se puede usar si lo convertís primero

# Si vas a usar postal_code como feature, convierte y descomenta esta línea:
# df2['postal_code'] = df2['postal_code'].fillna(0).astype(int)

# --------- 2. Normalizar ---------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# --------- 3. Entrenar modelo ---------
classifier = LogisticRegression(max_iter=3000)
classifier.fit(X_train_scaled, y_train)



LogisticRegression(max_iter=3000)

In [86]:
#X_test_scaled = scaler.transform(test_reviews)
common_cols = list(set(X_train.columns) & set(test_reviews.columns))
X_test = test_reviews[common_cols]
X_test_scaled = scaler.fit_transform(X_test)

X_test_scaled
predicted_stars = classifier.predict(X_test_scaled)
predicted_stars

submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})

# Mostrar el DataFrame de submission
submission_df.to_csv('submission.csv', index=False)

## ------------------------------------------

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from joblib import Parallel, delayed

# Download necessary datasets
nltk.download('punkt')

def preprocess_text_parallel(text):
    return word_tokenize(text.lower())

train_reviews['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in df['text'])
test_reviews['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in test_reviews['text'])
